<h1>Investigating Aggregate Factors Associated with Car Insurance Prices</h1>

Group members:
Karina Li,
Daisy Liu,
Xinyu Li,
Catherine Yang

<h1>Intro</h1>

Brief introduction of the topic & variables used for analysis

**RESEARCH QUESTIONS (DRAFT)**

RESEARCH QUESTION: What aggregate factors contribute to the average insurance costs?

Although car insurance costs vary between individuals based on their attributes (eg.
 age, experience, etc.) to what extent are average car insurance costs correlate with more general statistics (eg. the amount of bad drivers in a state or the amount of car thefts that goes on in a state)?

Further questions for investigation:
Does per capita bad drivers / accidents / theft etc. correlate with higher insurance prices more than the nominal values of these variables?
Do insurance firms adjust their perception of risk based on the concentration of events in the area or do they make decisions based on the raw number occurrences of these factors?
What are the different impacts on full insurance coverage price / minimum insurance coverage price?
Hypothesis: the minimum insurance coverage price may be less influenced by these negative factors because they are less of a risk, verify this hypothesis with regression


**OUTLINE OF PROJECT CONTENT (DRAFT)**

-Mapping for insurance data (visualization - maybe implement interactivity)


-Scatter plot between average insurance prices and the identified variables


-Regression using the identified variables, prediction analysis

Extension:
Doing separate analysis for maximum coverage & minimum coverage price, to see if the identified variables influence minimum coverage price less than maximum coverage price


<h1>Loading Packages & Datasets</h1>

In [59]:
! pip install fiona geopandas xgboost gensim pyLDAvis descartes mapclassify opendatasets 

In [14]:
! pip install geopandas opendatasets

In [26]:
! pip install folium

In [57]:
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import folium
from shapely.geometry import Point

%matplotlib inline

import numpy as np
import opendatasets as od


ModuleNotFoundError: No module named 'JSON'

In [17]:
#set seed
np.random.seed(123)

In [18]:
#load datasets

bad_drivers = pd.read_csv("https://raw.githubusercontent.com/mish-salt/323-Group-Project/main/bad-drivers.csv").set_index("State")
#bad drivers by state dataset

insurance_cost = pd.read_csv("https://raw.githubusercontent.com/mish-salt/323-Group-Project/main/auto_insurance_cost.csv").set_index("state")
#insurance cost by state dataset

state_crime = pd.read_csv("https://raw.githubusercontent.com/mish-salt/323-Group-Project/main/state_crime.csv")
#Crimes by state - includes a column for car theft



In [19]:
# #The dataset for car accidents is too large - here we import it directly from Kaggle
accident_data = 'https://www.kaggle.com/datasets/sobhanmoosavi/us-accidents'
od.download(accident_data) #download dataset file

#load in dataset
us_accidents = pd.read_csv('us-accidents/US_Accidents_March23.csv', usecols = ["State", "Severity"]) #The dataset is too large - select only the variables of interest


Skipping, found downloaded files in "./us-accidents" (use force=True to force download)


<h1>Viewing Datasets</h1>

In [20]:
bad_drivers.head()

,Number of drivers involved in fatal collisions per billion miles,Percentage Of Drivers Involved In Fatal Collisions Who Were Speeding,Percentage Of Drivers Involved In Fatal Collisions Who Were Alcohol-Impaired,Percentage Of Drivers Involved In Fatal Collisions Who Were Not Distracted,Percentage Of Drivers Involved In Fatal Collisions Who Had Not Been Involved In Any Previous Accidents,Car Insurance Premiums ($),Losses incurred by insurance companies for collisions per insured driver ($)
State,,,,,,,
Alabama,18.8,39,30,96,80,784.55,145.08
Alaska,18.1,41,25,90,94,1053.48,133.93
Arizona,18.6,35,28,84,96,899.47,110.35
Arkansas,22.4,18,26,94,95,827.34,142.39
California,12.0,35,28,91,89,878.41,165.63


In [21]:
insurance_cost.head()

,rank,full_coverage,minimum_coverage,difference
state,,,,
Michigan,1,8723,5282,3441
Rhode Island,2,3847,1589,2258
Louisiana,3,3525,1329,2196
Kentucky,4,3418,1338,2079
Florida,5,3370,2565,805


In [22]:
state_crime.head()

,State,Year,Data.Population,Data.Rates.Property.All,Data.Rates.Property.Burglary,Data.Rates.Property.Larceny,Data.Rates.Property.Motor,Data.Rates.Violent.All,Data.Rates.Violent.Assault,Data.Rates.Violent.Murder,...,Data.Rates.Violent.Robbery,Data.Totals.Property.All,Data.Totals.Property.Burglary,Data.Totals.Property.Larceny,Data.Totals.Property.Motor,Data.Totals.Violent.All,Data.Totals.Violent.Assault,Data.Totals.Violent.Murder,Data.Totals.Violent.Rape,Data.Totals.Violent.Robbery
0,Alabama,1960,3266740,1035.4,355.9,592.1,87.3,186.6,138.1,12.4,...,27.5,33823,11626,19344,2853,6097,4512,406,281,898
1,Alabama,1961,3302000,985.5,339.3,569.4,76.8,168.5,128.9,12.9,...,19.1,32541,11205,18801,2535,5564,4255,427,252,630
2,Alabama,1962,3358000,1067.0,349.1,634.5,83.4,157.3,119.0,9.4,...,22.5,35829,11722,21306,2801,5283,3995,316,218,754
3,Alabama,1963,3347000,1150.9,376.9,683.4,90.6,182.7,142.1,10.2,...,24.7,38521,12614,22874,3033,6115,4755,340,192,828
4,Alabama,1964,3407000,1358.7,466.6,784.1,108.0,213.1,163.0,9.3,...,29.1,46290,15898,26713,3679,7260,5555,316,397,992


In [ ]:
us_accidents.head()

<h1>Data Wrangling</h1>

For visualizing the correlation between these variables & insurance prices

In [28]:
#merging bad driver data with insurance data
df1 = pd.merge(insurance_cost, bad_drivers, left_on = "state", right_index = True)
#clean the crime dataset, use only obeservations from 2019, the closest time to 2020 in the dataset
crime_data = state_crime.loc[state_crime.Year == 2019]


#continue to merge it with the rate of stolen cars from the crime dataset


df2 = pd.merge(df1, crime_data, left_on = "state", right_on = "State").set_index("State")



#clean up the dataset - exclude columns that are not needed 
col_list = ["full_coverage",
            "minimum_coverage", 
            "difference", 
            "Number of drivers involved in fatal collisions per billion miles",
           "Percentage Of Drivers Involved In Fatal Collisions Who Were Speeding",
           "Percentage Of Drivers Involved In Fatal Collisions Who Were Alcohol-Impaired",
           "Data.Rates.Property.Motor"]
df3 = df2.loc[:, col_list]
#changing some variable names

plotting_data = df3.rename(columns = {"Data.Rates.Property.Motor":"stolen_cars_per_100k"}) #the variable refers to the number of stolen cars per 100,000 population

plotting_data.head()

,full_coverage,minimum_coverage,difference,Number of drivers involved in fatal collisions per billion miles,Percentage Of Drivers Involved In Fatal Collisions Who Were Speeding,Percentage Of Drivers Involved In Fatal Collisions Who Were Alcohol-Impaired,stolen_cars_per_100k
State,,,,,,,
Michigan,8723,5282,3441,14.1,24,28,177.7
Rhode Island,3847,1589,2258,11.1,34,38,128.2
Louisiana,3525,1329,2196,20.5,35,33,230.5
Kentucky,3418,1338,2079,21.4,19,23,228.2
Florida,3370,2565,805,17.9,21,29,181.8


<h1>Mapping Insurance Prices by State</h1>

In [132]:
m = folium.Map(locations = [35.3, -97.6], tiles = "CartoDB positron", zoom_start = 8, min_zoom = 2) #initializing interactive map

#mapping US insurance data - color legend referrs to values in the "full_coverage" column in insurance data
insurance_map = folium.Choropleth(geo_data = "./us-states.json",
                 data = insurance_cost,
                 columns = [insurance_cost.index, "full_coverage"],
                 key_on = "feature.properties.name",
                 fill_color = "YlGnBu",
                 name = "Full Coverage",
                 legend_name = "Full Coverage Insurance Price (USD)",
                bins = list(range(1000, 9500, 200)),
                 fill_opacity = 0.8).add_to(m)


In [134]:
#adding interactive popups that provide more information 
popup = folium.GeoJsonPopup(
    fields=["state"],
    aliases=["State"],
    localize=True,
    labels=True,
    style="background-color: yellow;")

tooltip = folium.GeoJsonTooltip(
    fields=["state"],
    aliases=["State:"],
    localize=True,
    sticky=False,
    labels=True,
    style="""
        background-color: #F0EFEF;
        border: 1px solid black;
        border-radius: 3px;
        box-shadow: 3px;
    """,
    max_width=800)



In [135]:
g = folium.GeoJson()

TypeError: GeoJson.__init__() missing 1 required positional argument: 'data'

<h1>Visualizing Correlation between Independent Variables and State Average Insurance Prices</h1>

<h1>Regression Analysis</h1>

<h1>Conclusion</h1>

<h1>References</h1>